1-	Consider the data set Exercise2DataSet. The dataset includes 14 features that represents clinical conditions of 500 ICU patients and target variable death that represents whether the patient died (=1) in the ICU or discharged alive (=0). You decided to use a K-neighbors classifier classification model to predict the death but worry about the choice of the neighbor size. (Before starting the analysis convert categorical features, into numerical features using get dummies function)

(a) Split your dataset into a test set and train set. (20% test set, 80% training set, use stratified sampling).
(b) Try different neighborhood sizes from 1 to 10 and, report LOOCV validation accuracies of each KNN model. 
(c) After picking the neighborhood size that has the highest LOOCV, refit your model on the standardized training set with the selected neighbor size and compute the test accuracy, and create a confusion matrix.
(d) Try different neighbor sizes from 1 to 10 and report 5-fold cross validation accuracies of each KNN model.
(e) After picking the neighbor size that has the highest 5-fold cross validation accuracy (if there is a tie, pick one of them), compute the test accuracy and create a confusion matrix.
(f) Which class is easier to predict for your model, death=0, or death=1? 

Do not forget to standardize your features only (do not standardize target variables). Hint: Use pipeline to prevent data leakage during cross validation.

2-	Now, by using the same dataset you will fit different regularized logistic regression models for the same purpose with different penalties.
(a) Split your dataset into a test set and train set. (20% test set, 80% t	raining set, use stratified sampling).
(b) Set the penalty to “l2” and try different C values [0.001,0.01,0.1,1,10,100,1000,10000] and report 10-fold cross validation accuracies. (You may need to increase the max_iter in case you end up with convergence issues).
(c) After picking the C values that has the highest 10-fold cross validation accuracy, compute the test accuracy, and generate the confusion matrix. Is logistic regression better than KNN?
(d) Repeat (b) to (c) by now setting penalty to “l1”. Are some of the features excluded from your final model?

Do not forget to standardize your features only (do not standardize target variables). Hint: Use pipeline to prevent data leakage during cross validation.




In [4]:
import pandas as pd
import numpy as np

In [5]:
# Read data
df = pd.read_csv("Exercise2DataSet.csv")

In [6]:
# 5 lines of the data
df.head()

,age,LOS,previous_LOS,previous_ICU_stays,cvc_status,SIRS_48_hour,MV_24_hour,Initial_SOFA,Discharge_SOFA,Max_SOFA,AdmitApache,DischargeApache,sex,Type,death
0,86,160.983333,1.800000,0,0,0,0,6,3,6,66,57,F,Surgical,0
1,61,103.533333,11.433333,0,1,1,0,7,4,7,80,73,F,Surgical,0
2,22,572.383333,14.450000,0,1,1,0,10,4,12,90,74,F,Surgical,0
3,58,51.200000,0.000000,0,1,1,1,7,3,7,78,64,M,Medical,0
4,18,35.116667,0.000000,0,0,1,1,7,2,7,73,49,M,Medical,0


In [12]:
# Value counts of the feature 'Type'

df.Type.value_counts()

Type
Surgical    315
Medical     185
Name: count, dtype: int64

In [13]:
# Checking for null values

df.isnull().sum()

age                   0
LOS                   0
previous_LOS          0
previous_ICU_stays    0
cvc_status            0
SIRS_48_hour          0
MV_24_hour            0
Initial_SOFA          0
Discharge_SOFA        0
Max_SOFA              0
AdmitApache           0
DischargeApache       0
sex                   0
Type                  0
death                 0
dtype: int64

In [14]:
# Info of the all features

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   age                 500 non-null    int64  
 1   LOS                 500 non-null    float64
 2   previous_LOS        500 non-null    float64
 3   previous_ICU_stays  500 non-null    int64  
 4   cvc_status          500 non-null    int64  
 5   SIRS_48_hour        500 non-null    int64  
 6   MV_24_hour          500 non-null    int64  
 7   Initial_SOFA        500 non-null    int64  
 8   Discharge_SOFA      500 non-null    int64  
 9   Max_SOFA            500 non-null    int64  
 10  AdmitApache         500 non-null    int64  
 11  DischargeApache     500 non-null    int64  
 12  sex                 500 non-null    object 
 13  Type                500 non-null    object 
 14  death               500 non-null    int64  
dtypes: float64(2), int64(11), object(2)
memory usage: 58.7+ K

In [15]:
# Define of the depended and independed values

X = df.drop("death", axis = 1)
y = df["death"]

In [19]:
# Display five lines of the X

X.head()

,age,LOS,previous_LOS,previous_ICU_stays,cvc_status,SIRS_48_hour,MV_24_hour,Initial_SOFA,Discharge_SOFA,Max_SOFA,AdmitApache,DischargeApache
0,86,160.983333,1.800000,0,0,0,0,6,3,6,66,57
1,61,103.533333,11.433333,0,1,1,0,7,4,7,80,73
2,22,572.383333,14.450000,0,1,1,0,10,4,12,90,74
3,58,51.200000,0.000000,0,1,1,1,7,3,7,78,64
4,18,35.116667,0.000000,0,0,1,1,7,2,7,73,49


In [17]:
# Dropping categorical values in the X

X.drop(X[["sex", "Type"]], axis = 1, inplace = True)

In [18]:
#-1-
# Create dummy/indicator variables for 'sex' and 'Type' columns and convert them to integer type
dummies = pd.get_dummies(df[["sex", "Type"]], drop_first = True ).astype("int")


In [20]:
# Concatenate the original DataFrame X with the dummies DataFrame along the columns (axis=1)
X = pd.concat([X, dummies], axis=1, ignore_index=False)

# Display the first few rows of the concatenated DataFrame
X.head()

,age,LOS,previous_LOS,previous_ICU_stays,cvc_status,SIRS_48_hour,MV_24_hour,Initial_SOFA,Discharge_SOFA,Max_SOFA,AdmitApache,DischargeApache,sex_M,Type_Surgical
0,86,160.983333,1.800000,0,0,0,0,6,3,6,66,57,0,1
1,61,103.533333,11.433333,0,1,1,0,7,4,7,80,73,0,1
2,22,572.383333,14.450000,0,1,1,0,10,4,12,90,74,0,1
3,58,51.200000,0.000000,0,1,1,1,7,3,7,78,64,1,0
4,18,35.116667,0.000000,0,0,1,1,7,2,7,73,49,1,0


In [22]:
#-1-a-
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [24]:
from sklearn.preprocessing import StandardScaler

# Create an instance of StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train)

# Transform the training data using the fitted scaler
X_train = scaler.transform(X_train)

# Transform the test data using the same scaler
X_test = scaler.transform(X_test)

In [43]:
#-1-b-
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier

scores = []

# Loop through neighbor values from 1 to 10
for i in range(1, 11):
    # Create a pipeline with StandardScaler and KNeighborsClassifier with the current number of neighbors
    knn_model = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors = i))
    
    # Perform leave-one-out cross-validation and calculate accuracy scores
    scoreCV = cross_val_score(knn_model, X_train, y_train, scoring = "accuracy", cv = LeaveOneOut())
    
    # Append the mean cross-validation score and the number of neighbors to the scores list
    scores.append([np.mean(scoreCV), i])
    
# Create a DataFrame from the scores list with columns "scores" and "Neighbors"
df = pd.DataFrame(scores, columns=["scores", "Neighbors"])

# Display the DataFrame
df

,scores,Neighbors
0,0.8425,1
1,0.8550,2
2,0.8800,3
3,0.8700,4
4,0.8700,5
5,0.8725,6
6,0.8800,7
7,0.8775,8
8,0.8850,9
9,0.8800,10


In [54]:
#-1-c-
# Final model.
# Create a pipeline with StandardScaler and KNeighborsClassifier with the current number of neighbors
knn_model = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors = 7))
    
# Perform leave-one-out cross-validation and calculate accuracy scores
scoreCV = cross_val_score(knn_model, X_train, y_train, scoring = "accuracy", cv = LeaveOneOut())


In [140]:
# Fit the KNN model on the training data
knn_model.fit(X_train, y_train)

# Use the fitted model to predict labels for the test data
y_pred = knn_model.predict(X_test)

# Calculate the accuracy score by comparing predicted labels with true labels (y_test)
accuracy = accuracy_score(y_pred, y_test)

# Print the accuracy score
print("Accuracy Score LOOCV:", accuracy)


Accuracy Score LOOCV: 0.95


In [125]:
model_accuracy = pd.DataFrame( columns = ["Accuracy"])

model_accuracy.loc["KNN LOOCV"] = [accuracy]

model_accuracy

,Accuracy
KNN LOOCV,0.92


In [56]:
# Compute the confusion matrix by comparing true labels (y_test) with predicted labels (y_pred)
matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
matrix

array([[82,  0],
       [ 5, 13]], dtype=int64)

In [59]:
#-1-d-
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier

scores = []

# Loop through neighbor values from 1 to 10
for i in range(1, 11):
    # Create a pipeline with StandardScaler and KNeighborsClassifier with the current number of neighbors
    knn_model = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=i))
    
    # Perform kfold cross-validation and calculate accuracy scores
    scoreCV = cross_val_score(knn_model, X_train, y_train, scoring = "accuracy", 
                              cv = KFold(n_splits = 5, random_state = 1, shuffle = True))
    
    # Append the mean cross-validation score and the number of neighbors to the scores list
    scores.append([np.mean(scoreCV), i])
    
# Create a DataFrame from the scores list with columns "scores" and "Neighbors"
df = pd.DataFrame(scores, columns=["scores", "Neighbors"])

# Display the DataFrame
df

,scores,Neighbors
0,0.8525,1
1,0.8525,2
2,0.8625,3
3,0.8550,4
4,0.8675,5
5,0.8700,6
6,0.8725,7
7,0.8750,8
8,0.8750,9
9,0.8775,10


In [63]:
#-1-e-
# Final model.
# Create a pipeline with StandardScaler and KNeighborsClassifier with the current number of neighbors
knn_model = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors = 9))
    
# Perform leave-one-out cross-validation and calculate accuracy scores
scoreCV = cross_val_score(knn_model, X_train, y_train, scoring = "accuracy", 
                         cv = KFold(n_splits = 5, random_state = 1, shuffle = True))


In [139]:
# Fit the KNN model on the training data
knn_model.fit(X_train, y_train)

# Use the fitted model to predict labels for the test data
y_pred = knn_model.predict(X_test)

# Calculate the accuracy score by comparing predicted labels with true labels (y_test)
accuracykfold = accuracy_score(y_pred, y_test)

# Print the accuracy score
print("Accuracy Score KFold:", accuracykfold)


Accuracy Score KFold: 0.95


In [128]:
model_accuracy.loc["KNN KFold"] = [accuracykfold]

model_accuracy

,Accuracy
KNN LOOCV,0.92
KNN KFold,0.95


In [62]:
# Compute the confusion matrix by comparing true labels (y_test) with predicted labels (y_pred)
matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
matrix

array([[82,  0],
       [ 5, 13]], dtype=int64)

In [71]:
# -2-b-

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

scores = []

# Create a list of regularization parameters as powers of 10
#[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
reg_c = [10**i for i in range(-3, 5)]

# Loop through each regularization parameter in the reg_c list
for c in reg_c:    
    # Create a pipeline with StandardScaler and LogisticRegression with the current regularization parameter
    logmodel = make_pipeline(StandardScaler(), 
                             LogisticRegression(max_iter = 1000, C = c, penalty = "l2", solver = 'liblinear')) 
    
    # Perform cross-validation with 10 folds and calculate the accuracy score for each fold
    scoreCV = cross_val_score(logmodel, X_train, y_train, scoring = "accuracy", 
                              cv = KFold(n_splits = 10, random_state = 1, shuffle = True))
    
    # Append the mean cross-validation score to the scores list
    scores.append([np.mean(scoreCV), c])
    
# Create a DataFrame from the scores list with the column name "scores"
df = pd.DataFrame(scores, columns = ["scores l2", "reg_c"])

# Display the DataFrame
df


,scores l2,reg_c
0,0.8625,0.001
1,0.9025,0.010
2,0.8975,0.100
3,0.8925,1.000
4,0.8925,10.000
5,0.8925,100.000
6,0.8925,1000.000
7,0.8925,10000.000


In [35]:
# final model
# Create a pipeline with StandardScaler and LogisticRegression with the current regularization parameter
# Perform cross-validation with 10 folds and calculate the accuracy score for each fold
logmodel = make_pipeline(StandardScaler(), 
                        LogisticRegression(max_iter = 1000, C = 0.01, penalty = "l2", solver = 'liblinear')) 
scoreCV = cross_val_score(logmodel, X_train, y_train, scoring = "accuracy", 
                        cv = KFold(n_splits = 10, random_state = 1, shuffle = True))


In [77]:
# Fit the logistic regression model on the training data
logmodel.fit(X_train, y_train)

# Use the fitted model to predict labels for the test data
y_pred = logmodel.predict(X_test)

# Calculate the accuracy score by comparing predicted labels with true labels (y_test)
accuracy = accuracy_score(y_pred, y_test)

# Print the accuracy score
print("Accuracy Score:", accuracy)

Accuracy Score: 0.82


In [137]:
# Logistic Regression modelini tanımlama
logmodel = LogisticRegression(C=0.1, penalty='l2', max_iter=1000, solver='liblinear')

# Modeli eğitme
logmodel.fit(X_train, y_train)

# Test verisi üzerinde tahmin yapma
y_pred = logmodel.predict(X_test)

# Doğruluk skorunu hesaplama
accuracyl2 = accuracy_score(y_pred, y_test)

# Doğruluk skorunu yazdırma
print("Accuracy with L2 penalty:", accuracyl2)

Accuracy with L2 penalty: 0.91


In [131]:
model_accuracy.loc["LogisticRegression l2"] = [accuracyl2]

model_accuracy

,Accuracy
KNN LOOCV,0.92
KNN KFold,0.95
LogisticRegression l2,0.91


In [78]:
# Compute the confusion matrix by comparing true labels (y_test) with predicted labels (y_pred)
matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
matrix

array([[82,  0],
       [18,  0]], dtype=int64)

In [141]:
from sklearn.metrics import roc_auc_score

y_pred_proba = logmodel.predict_proba(X_test)[:, 1]

# ROC AUC 
auc_score = roc_auc_score(y_test, y_pred_proba)

print("ROC AUC Score:", auc_score)

ROC AUC Score: 0.9512195121951219


In [84]:
# -2-c-

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
scores = []

#[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
reg_c = [10**i for i in range(-3, 5)]

# Loop through each regularization parameter in the reg_c list
for c in reg_c:    
    # Create a pipeline with StandardScaler and LogisticRegression with the current regularization parameter
    logmodel = make_pipeline(StandardScaler(), 
                             LogisticRegression(max_iter = 1000, C = c, penalty = "l1", solver = 'liblinear')) 
    
    # Perform cross-validation with 10 folds and calculate the accuracy score for each fold
    scoreCV = cross_val_score(logmodel, X_train, y_train, scoring = "accuracy", 
                              cv = KFold(n_splits = 10, random_state = 1, shuffle = True))
    
    # Append the mean cross-validation score to the scores list
    scores.append([np.mean(scoreCV), c])
    
# Create a DataFrame from the scores list with the column name "scores"
df = pd.DataFrame(scores, columns = ["scores l1", "reg_c"])

# Display the DataFrame
df


,scores l1,reg_c
0,0.8050,0.001
1,0.8050,0.010
2,0.9000,0.100
3,0.8925,1.000
4,0.8925,10.000
5,0.8925,100.000
6,0.8925,1000.000
7,0.8925,10000.000


In [88]:
# final model
# Create a pipeline with StandardScaler and LogisticRegression with the current regularization parameter
# Perform cross-validation with 10 folds and calculate the accuracy score for each fold
logmodel = make_pipeline(StandardScaler(), 
                        LogisticRegression(max_iter = 1000, C = 0.1, penalty = "l1", solver = 'liblinear')) 
scoreCV = cross_val_score(logmodel, X_train, y_train, scoring = "accuracy", 
                        cv = KFold(n_splits = 10, random_state = 1, shuffle = True))

In [138]:
# Fit the logistic regression model on the training data
logmodel.fit(X_train, y_train)

# Use the fitted model to predict labels for the test data
y_pred = logmodel.predict(X_test)

# Calculate the accuracy score by comparing predicted labels with true labels (y_test)
accuracyl1 = accuracy_score(y_pred, y_test)

# Print the accuracy score
print("Accuracy with L1 penalty:", accuracyl1)


Accuracy with L1 penalty: 0.91


In [133]:
model_accuracy.loc["LogisticRegression l1"] = [accuracyl2]

model_accuracy

,Accuracy
KNN LOOCV,0.92
KNN KFold,0.95
LogisticRegression l2,0.91
LogisticRegression l1,0.91


In [90]:
# Compute the confusion matrix by comparing true labels (y_test) with predicted labels (y_pred)
matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
matrix

array([[81,  1],
       [ 5, 13]], dtype=int64)